# DAVID RODRIGUES ALBUQUERQUE - Tarefa Final - Snake
### Introdução ao Aprendizado de Máquina - 2023.1 - Ciência da Computação

Feito por:

## David Rodrigues Albuquerque - 120047390

---

In [17]:
%pip install pygame
import random
import numpy as np
import pygame
import sys

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\david\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [18]:
class Snake:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.body = [(width // 2, height // 2)]
        self.direction = 'up'
        self.apple = self.generate_apple()

    def generate_apple(self):
        while True:
            apple = (random.randint(0, self.width - 1),
                     random.randint(0, self.height - 1))
            if apple not in self.body:
                return apple

    def change_direction(self, direction):
        if direction == 'up' and self.direction != 'down':
            self.direction = 'up'
        elif direction == 'down' and self.direction != 'up':
            self.direction = 'down'
        elif direction == 'left' and self.direction != 'right':
            self.direction = 'left'
        elif direction == 'right' and self.direction != 'left':
            self.direction = 'right'

    def move(self):
        head = self.body[0]
        if self.direction == 'up':
            new_head = (head[0], (head[1] - 1))
        elif self.direction == 'down':
            new_head = (head[0], (head[1] + 1))
        elif self.direction == 'left':
            new_head = ((head[0] - 1), head[1])
        elif self.direction == 'right':
            new_head = ((head[0] + 1), head[1])

        if (
            new_head[0] < 0 or new_head[0] >= self.width or
            new_head[1] < 0 or new_head[1] >= self.height or
            new_head in self.body
        ):
            return True

        self.body.insert(0, new_head)
        if new_head == self.apple:
            self.apple = self.generate_apple()
        else:
            self.body.pop()

    def check_collision(self):
        head = self.body[0]
        if (
            head[0] < 0 or head[0] >= self.width or
            head[1] < 0 or head[1] >= self.height or
            head in self.body[1:]
        ):
            return True
        return False


In [19]:
class Game:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.snake = Snake(width, height)
        self.game_over = False

        pygame.init()
        self.clock = pygame.time.Clock()
        self.window = pygame.display.set_mode((width * 20, height * 20))
        pygame.display.set_caption('Snake Game')

    def run(self):
        while True:
            # Define a taxa de atualização do jogo (frames por segundo)
            self.clock.tick(15)
            self.handle_events()
            self.update()
            self.render()
            if self.game_over:
                self.show_game_over_menu()

    def handle_events(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                self.quit()
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_UP:
                    self.snake.change_direction('up')
                elif event.key == pygame.K_DOWN:
                    self.snake.change_direction('down')
                elif event.key == pygame.K_LEFT:
                    self.snake.change_direction('left')
                elif event.key == pygame.K_RIGHT:
                    self.snake.change_direction('right')

    def update(self):
        if self.snake.move():
            self.game_over = True

    def render(self):
        self.window.fill((0, 0, 0))  # Preenche a janela com a cor preta
        for segment in self.snake.body:
            # Desenha cada segmento da cobra
            pygame.draw.rect(self.window, (255, 255, 255),
                             (segment[0] * 20, segment[1] * 20, 20, 20))
        pygame.draw.rect(self.window, (255, 0, 0), (
            self.snake.apple[0] * 20, self.snake.apple[1] * 20, 20, 20))  # Desenha a maçã
        pygame.display.flip()  # Atualiza a janela

    def show_game_over_menu(self):
        font = pygame.font.SysFont(None, 36)
        text = font.render('Game Over!', True, (255, 0, 0))
        self.window.blit(text, (self.width * 10 - text.get_width() //
                         2, self.height * 10 - text.get_height() // 2))
        text = font.render('Press R to restart or Q to quit',
                           True, (255, 255, 255))
        self.window.blit(text, (self.width * 10 - text.get_width() //
                         2, self.height * 10 + text.get_height() // 2 + 10))
        pygame.display.flip()

        while True:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.quit()
                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_r:
                        self.reset()
                        return
                    elif event.key == pygame.K_q:
                        self.quit()

    def reset(self):
        self.snake = Snake(self.width, self.height)
        self.game_over = False

    def quit(self):
        pygame.quit()
        sys.exit()


In [20]:
# Executando o jogo
game = Game(30, 30)  # Define a largura e altura do grid do jogo

while True:
    game.run()
    if not game.game_over:
        break


SystemExit: 

# Objetivo


# Referências Bibliográficas
